<a href="https://colab.research.google.com/github/HatemMoushir/smart-ai-assistant/blob/main/sentiment140_stanford_only.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# ✅ تدريب Sentiment140 فقط - من Stanford - باستخدام DistilBERT

# 1. تحميل البيانات الأصلية من Stanford
!wget https://cs.stanford.edu/people/alecmgo/trainingandtestdata.zip
!unzip trainingandtestdata.zip

# 2. قراءة البيانات
import pandas as pd

cols = ['label', 'id', 'date', 'query', 'user', 'text']
df = pd.read_csv('training.1600000.processed.noemoticon.csv', encoding='latin-1', names=cols)

# 3. تحويل التصنيفات:
# 0 → سلبي | 4 → إيجابي → نحولها لـ 1
df = df[df['label'].isin([0, 4])]
df['label'] = df['label'].map({0: 0, 4: 1})

# 4. تقليل الحجم لعينة صغيرة للتجربة
df_small = df.sample(10000, random_state=42).reset_index(drop=True)

# 5. تحويل إلى Dataset
from datasets import Dataset
dataset = Dataset.from_pandas(df_small[['text', 'label']])

# 6. تحميل Tokenizer & Model
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer
import numpy as np
import evaluate

tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")

def tokenize(example):
    return tokenizer(example["text"], truncation=True, padding="max_length")

dataset = dataset.map(tokenize, batched=True)

# 7. تقسيم بيانات التدريب والتقييم
dataset = dataset.train_test_split(test_size=0.1)

# 8. تحميل الموديل
model = AutoModelForSequenceClassification.from_pretrained("distilbert-base-uncased", num_labels=2)

# 9. الإعدادات
accuracy = evaluate.load("accuracy")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return accuracy.compute(predictions=predictions, references=labels)

args = TrainingArguments(
    output_dir="sent140_model",
    evaluation_strategy="epoch",
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=2,
    save_strategy="epoch",
    load_best_model_at_end=True,
    logging_steps=100,
)

trainer = Trainer(
    model=model,
    args=args,
    train_dataset=dataset["train"],
    eval_dataset=dataset["test"],
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)

# 🔁 تدريب الموديل
trainer.train()